In [1]:
import gym
from itertools import chain, combinations
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical

<h2>Use CUDA</h2>

In [2]:
use_cuda = torch.cuda.is_available()
device   = torch.device("cuda" if use_cuda else "cpu")
print(device)

cuda


<h2>Create Environments</h2>

In [3]:
import sys
sys.path.append('/home/deepak/Desktop/RL-Adventure-2/')
from common import SubprocVecEnv

num_envs = 16
env_name = "CartPole-v1"

def make_env():
    def _thunk():
        env = gym.make(env_name)
        return env

    return _thunk

envs = [make_env() for i in range(num_envs)]
envs = SubprocVecEnv(envs)

env = gym.make(env_name)

<h2>Neural Network</h2>

In [4]:
class ActorCritic(nn.Module):
    def __init__(self, num_inputs, num_outputs, hidden_size, std=0.0):
        super(ActorCritic, self).__init__()
        
        self.critic = nn.Sequential(
            nn.Linear(num_inputs, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1)
        )
        
        self.actor = nn.Sequential(
            nn.Linear(num_inputs, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, num_outputs),
            nn.Softmax(dim=1),
        )
        
    def forward(self, x):
        value = self.critic(x)
        probs = self.actor(x)
        dist  = Categorical(probs)
        return dist, value

In [5]:
def test_env(model, transform):
    state = transform(env.reset())
    done = False
    total_reward = 0
    while not done:
        state = torch.FloatTensor(state).unsqueeze(0).to(device)
        dist, _ = model(state)
        next_state, reward, done, _ = env.step(dist.sample().cpu().numpy()[0])
        state = transform(next_state)
        total_reward += reward
    return total_reward

In [6]:
def compute_returns(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

In [7]:

def ith_powerset(n, i):
    s = list(range(n))
    return list(chain.from_iterable(combinations(s, r) for r in range(1, len(s)+1)))[i]

ith_powerset(4, 8)

(1, 3)

In [8]:
class Hparams:
    num_inputs = envs.observation_space.shape[0]
    num_outputs = envs.action_space.n
    hidden_size = 256
    lr = 1e-3
    num_steps = 5
    max_frames = 25000


In [9]:
def get_transform(indices):
    def transform(x):
        return [x[idx] for idx in indices]
    return transform

In [10]:
def train_model(hparams, indices):
    transform = lambda x: x#get_transform(indices)
    
    # init model and optimizer
    model = ActorCritic(
        num_inputs=hparams.num_inputs,
        num_outputs=hparams.num_outputs,
        hidden_size=hparams.hidden_size,
    ).to(device)

    optimizer = optim.Adam(lr=hparams.lr, params=model.parameters())

    # init env
    frame_idx = 0
    state = transform(envs.reset())

    while frame_idx < hparams.max_frames:

        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0

        for _ in range(hparams.num_steps):
            state = torch.FloatTensor(state).to(device)
            dist, value = model(state)

            action = dist.sample()
            next_state, reward, done, _ = envs.step(action.cpu().numpy())
            next_state = transform(next_state)

            log_prob = dist.log_prob(action)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.FloatTensor(reward).unsqueeze(1).to(device))
            masks.append(torch.FloatTensor(1 - done).unsqueeze(1).to(device))

            state = next_state
            frame_idx += 1

            if frame_idx % 2500 == 0:
                # average reward over 10 episodes
                avg_reward = np.mean([test_env(model, transform) for _ in range(10)])
                print(f"Frame={frame_idx} => avg_reward={avg_reward:.4f}")

        next_state = torch.FloatTensor(next_state).to(device)
        _, next_value = model(next_state)
        returns = compute_returns(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        loss = actor_loss + 0.5 * critic_loss - 0.001 * entropy

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    avg_reward = np.mean([test_env(model, transform) for _ in range(200)])
    return avg_reward


In [11]:
train_model(hparams=Hparams(), indices=[0,1,2,3])

Frame=2500 => avg_reward=61.7000
